In [1]:
import sys
import json
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [3]:
import sys
sys.path.insert(0, r'../../../quetzal')
from quetzal.model import stepmodel
from quetzal.io import excel
import pandas as pd
if manual:
    %matplotlib inline

In [4]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [5]:
sm = stepmodel.read_zippedpickles(model_folder +'los')
sm.segments = ['car_owner', 'pt_captive'] 

zones: 100%|██████████| 28/28 [00:13<00:00,  2.12it/s]              


In [6]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
kwargs = var['preparation_logit'].to_dict()
kwargs

{'time': 20,
 'price': 15,
 'mode': 1,
 'pt_mode': 0.5,
 'pt_path': 0.1,
 'transfers': 5}

In [7]:
sm.preparation_logit(
    segments=['car_owner', 'pt_captive'],
    **kwargs
)

In [8]:
sm.utility_values.T

value,time,price,ntransfers,mode_utility
segment,,,,
root,20,15,5,1
car_owner,20,15,5,1
pt_captive,20,15,5,1


In [9]:
sm.logit_scales.T 

route_type,bus,walk,root,car,tram,pt
segment,,,,,,
root,0.1,0.0,1.0,0.0,0.1,0.5
car_owner,0.1,0.0,1.0,0.0,0.1,0.5
pt_captive,0.1,0.0,1.0,0.0,0.1,0.5


In [10]:
sm.mode_utility.T

route_type,bus,walk,root,car,tram
segment,,,,,
root,0,0,0,0,0
car_owner,0,0,0,0,0
pt_captive,0,0,0,0,0


In [11]:
sm.mode_utility.loc[ 'car', 'pt_captive'] = -100
sm.mode_utility.T

route_type,bus,walk,root,car,tram
segment,,,,,
root,0,0,0,0,0
car_owner,0,0,0,0,0
pt_captive,0,0,0,-100,0


In [12]:
sm.mode_nests.T 

route_type,bus,walk,root,car,tram,pt
segment,,,,,,
root,pt,root,NaN,root,pt,root
car_owner,pt,root,NaN,root,pt,root
pt_captive,pt,root,NaN,root,pt,root


In [14]:
sm.pt_los['ntransfers'] = sm.pt_los['alighting_links'].apply(lambda x: max(len(x)-1, 0))
sm.pt_los['price'] = 0  ######
sm.car_los['ntransfers'] = 0
sm.car_los['price'] = 0

In [15]:
sm.analysis_pt_time(
        boarding_time=None,
        alighting_time=None,
        walk_on_road=True,
    )
 

sm.analysis_car_route_type() 

sm.los = pd.concat([sm.pt_los, sm.car_los]).reset_index(drop=True)
sm.los['path'] = sm.los['path'].apply(lambda p: tuple(p)) 


sm.analysis_mode_utility()

100%|██████████| 2/2 [00:00<00:00,  8.41it/s]


In [16]:
sm.initialize_logit()
sm.step_logit()

100%|██████████| 10/10 [00:02<00:00,  3.98it/s]


In [17]:
sm.compute_los_volume()

In [18]:
sm.pt_los = sm.los.copy()
sm.pt_los = sm.pt_los.loc[sm.pt_los['route_type'] != 'car']

In [19]:
sm.step_pt_assignment(
    road=True,
    boardings=True,
    alightings=True,
    transfers=True
 
)

pt_captive: 100%|██████████| 2/2 [00:17<00:00,  8.65s/it]


In [20]:
sm.links.rename(columns={'load': 'volume'}, inplace=True)
sm.analysis_pt_length(
     walk_on_road=True
)
sm.analysis_car_length()
sm.analysis_pt_time(
     walk_on_road=True
)

In [21]:
sm.summary_link_max(inplace=True)
sm.summary_link_sum(inplace=True)
sm.summary_od(inplace=True)

In [22]:

sm.los = pd.concat([sm.pt_los, sm.car_los]).reset_index(drop=True)

In [23]:
sm.summary_path_average(inplace=True)
sm.summary_aggregated_path_average(inplace=True, pt_route_types=['subway', 'bus', 'tram', 'rail'])
sm.summary_path_sum(inplace=True)

s = sm.stack_link_sum.unstack([-1, -2]).groupby(level=0).sum().stack([0, 1])
s.name = 'sum'
sm.stack_aggregated_link_sum = s 
s = sm.stack_link_max.unstack().groupby(level=0).max().stack()
s.name = 'max'
sm.stack_aggregated_link_max = s

In [24]:
sm.od_los

segment                  car_owner                             \
sum                         volume price time in_vehicle_time   
origin  destination mode                                        
zone_0  zone_0      car   0.000000   0.0  0.0             0.0   
                    walk  1.000060   0.0  0.0             0.0   
        zone_1      bus   0.000000   0.0  0.0             0.0   
                    car   0.000000   0.0  0.0             0.0   
        zone_10     bus   0.000000   0.0  0.0             0.0   
...                            ...   ...  ...             ...   
zone_99 zone_97     walk  0.000000   0.0  0.0             0.0   
        zone_98     car   0.000000   0.0  0.0             0.0   
                    walk  0.000000   0.0  0.0             0.0   
        zone_99     car   0.000000   0.0  0.0             0.0   
                    walk  2.000129   0.0  0.0             0.0   

segment                                               pt_captive             \
sum                      in_vehicle_length ntransfers     volume price time   
origin  destination mode                                                      
zone_0  zone_0      car                0.0        0.0   0.000000   0.0  0.0   
                    walk               0.0        0.0   1.000060   0.0  0.0   
        zone_1      bus                0.0        0.0   0.000000   0.0  0.0   
                    car                0.0        0.0   0.000000   0.0  0.0   
        zone_10     bus                0.0        0.0   0.000000   0.0  0.0   
...                                    ...        ...        ...   ...  ...   
zone_99 zone_97     walk               0.0        0.0   0.000000   0.0  0.0   
        zone_98     car                0.0        0.0   0.000000   0.0  0.0   
                    walk               0.0        0.0   0.000000   0.0  0.0   
        zone_99     car                0.0        0.0   0.000000   0.0  0.0   
                    walk               0.0        0.0   1.500144   0.0  0.0   

segment                                                                \
sum                      in_vehicle_time in_vehicle_length ntransfers   
origin  destination mode                                                
zone_0  zone_0      car              0.0               0.0        0.0   
                    walk             0.0               0.0        0.0   
        zone_1      bus              0.0               0.0        0.0   
                    car              0.0               0.0        0.0   
        zone_10     bus              0.0               0.0        0.0   
...                                  ...               ...        ...   
zone_99 zone_97     walk             0.0               0.0        0.0   
        zone_98     car              0.0               0.0        0.0   
                    walk             0.0               0.0        0.0   
        zone_99     car              0.0               0.0        0.0   
                    walk             0.0               0.0        0.0   

segment                       root                             \
sum                         volume price time in_vehicle_time   
origin  destination mode                                        
zone_0  zone_0      car   0.000000   0.0  0.0             0.0   
                    walk  2.000120   0.0  0.0             0.0   
        zone_1      bus   0.000000   0.0  0.0             0.0   
                    car   0.000000   0.0  0.0             0.0   
        zone_10     bus   0.000000   0.0  0.0             0.0   
...                            ...   ...  ...             ...   
zone_99 zone_97     walk  0.000000   0.0  0.0             0.0   
        zone_98     car   0.000000   0.0  0.0             0.0   
                    walk  0.000000   0.0  0.0             0.0   
        zone_99     car   0.000000   0.0  0.0             0.0   
                    walk  3.500273   0.0  0.0             0.0   

segment                                         

In [25]:
sm.to_zippedpickles(model_folder + 'logit_assignment')

stack_aggregated_link_max: 100%|██████████| 47/47 [01:17<00:00,  1.65s/it]
